In [ ]:
✅ What this query is useful for

✅ Identify data stoppage
✅ Monitor onboarding sources
✅ SOC “not reporting” detection
✅ Can be used as alert

✅ Extra: If you want ONLY important sourcetypes (exclude internal)
| tstats latest(_time) as lastTime where index!=_internal earliest=-7d by index sourcetype
| eval gap=now()-lastTime
| where gap>900
| eval gap_min=round(gap/60,2)
| eval lastTime=strftime(lastTime,"%Y-%m-%d %H:%M:%S")
| table index sourcetype gap_min lastTime


Gap Detection (recommended query)

This finds missing buckets for each index+sourcetype.

🔥 Detect gaps > 15 mins in last 24h
| tstats count where index=* earliest=-24h by index sourcetype _time span=5m
| sort 0 index sourcetype _time
| streamstats current=f last(_time) as prevTime by index sourcetype
| eval gap_sec=_time-prevTime
| where gap_sec > 900
| eval gap_min=round(gap_sec/60,2)
| eval gap_start=strftime(prevTime,"%Y-%m-%d %H:%M:%S")
| eval gap_end=strftime(_time,"%Y-%m-%d %H:%M:%S")
| table index sourcetype gap_min gap_start gap_end
| sort - gap_min


✅ Output shows:

gap duration

gap start time

gap end time
for each index/sourcetype

    Fetching index and sourcetype

| tstats summariesonly=t allow_old_summaries=t count where (index=* OR index=_*) by index sourcetype
| rename count as Count index as Index sourcetype as Sourcetype